In [42]:
#Acess the project directory
import os
os.chdir(r"C:\Users\USER\Desktop\MLDefaults\End-to-End-Machine-Learning-with-Mlflow")

In [43]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/samaTech-sys/End-to-End-Machine-Learning-with-Mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "SamaTech-sys"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "27c279e0895284a35a6198b88076b93c13073cab"

In [44]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path 
    model_path: Path 
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [45]:
#importing all project paths and modules necessary for project configurations 
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [46]:
#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ELASTICNET
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path = config.test_data_path,
            model_path=config.model_path, 
            all_params= params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/samaTech-sys/End-to-End-Machine-Learning-with-Mlflow.mlflow"
        )
        return model_evaluation_config

In [47]:
import os 
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib
import mlflow
import mlflow.sklearn
from mlProject.utils.common import save_json


In [48]:
class ModelEvaluation: 
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred): 
        rmse = np.sqrt(mean_absolute_error(actual, pred))
        mse = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mse, r2

    def log_into_mlflow(self): 
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis =1)
        test_y = test_data[self.config.target_column]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mse, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            #saving metrics as local 
            scores = {"rmse": rmse, "mse": mse, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mse", mse)
            mlflow.log_metric("r2", r2)
            
            #model registry does notwork with filestore
            if tracking_url_type_store != "file":
                #Register the model 
                #There are other ways to use the model registry that depend on the use case
                #Please refer to the doc for more information 
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else: 
                mlflow.sklearn.log_model(model, "model")

In [49]:
#Update pipeline Step 
try: 
    config = ConfigurationManager()
    model_evaluation_config =config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-05-14 01:54:04,864: INFO: common: yaml file: C:\Users\USER\Desktop\MLDefaults\End-to-End-Machine-Learning-with-Mlflow\config\config.yaml loaded successfully]
[2025-05-14 01:54:04,881: INFO: common: yaml file: C:\Users\USER\Desktop\MLDefaults\End-to-End-Machine-Learning-with-Mlflow\params.yaml loaded successfully]
[2025-05-14 01:54:04,895: INFO: common: yaml file: C:\Users\USER\Desktop\MLDefaults\End-to-End-Machine-Learning-with-Mlflow\schema.yaml loaded successfully]
[2025-05-14 01:54:04,896: INFO: common: created directory at:artifacts]


[2025-05-14 01:54:04,907: INFO: common: created directory at:artifacts/model_evaluation]
[2025-05-14 01:54:05,614: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/05/14 01:54:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/05/14 01:54:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 5
Created version '5' of model 'ElasticnetModel'.


🏃 View run funny-stork-990 at: https://dagshub.com/samaTech-sys/End-to-End-Machine-Learning-with-Mlflow.mlflow/#/experiments/0/runs/7e15613da17948199607efca18c08cea
🧪 View experiment at: https://dagshub.com/samaTech-sys/End-to-End-Machine-Learning-with-Mlflow.mlflow/#/experiments/0
